In [ ]:
import pandas as pd
import numpy as np

Load the data

In [ ]:
def load_data():
    """Load stock data from a user-uploaded ZIP file containing a CSV."""
    uploaded = files.upload()
    zip_file_name = list(uploaded.keys())[0]
    with zipfile.ZipFile(io.BytesIO(uploaded[zip_file_name]), 'r') as z:
        csv_filename = [name for name in z.namelist() if name.endswith('.csv')][0]
        with z.open(csv_filename) as f:
            return pd.read_csv(f, parse_dates=['Date'], index_col='Date')

Caculate the indicators from the given data


*   Moving Averages (MAs): 10-day and 50-day moving averages of closing prices.
*   Relative Strength Index (RSI): Measures stock momentum over 14 days.
*   Volatility: Standard deviation of percentage changes in price over 20 days.
*   Average True Range (ATR): Measures price fluctuations over 14 days.



1.   Return: A DataFrame with additional indicator columns.





In [ ]:
def calculate_indicators(data):
    """Calculate trend indicators, volatility measures, and price patterns."""
    data['10-day MA'] = data['Close'].rolling(window=10).mean()
    data['50-day MA'] = data['Close'].rolling(window=50).mean()
    data['RSI'] = 100 - (100 / (1 + data['Close'].pct_change().rolling(window=14).mean() /
                                data['Close'].pct_change().rolling(window=14).std()))
    data['Volatility'] = data['Close'].pct_change().rolling(window=20).std()
    data['ATR'] = (data['High'] - data['Low']).rolling(window=14).mean()
    return data

Calculate the price movement



*   Next Day Return = Closing price difference between the next day and the current day.
*   If Next Day Return > 0, it's labeled as an Uptrend (1); otherwise, it's a Downtrend (0).



1.  Return: A DataFrame with movement labels





In [ ]:
def calculate_price_movement(data):
    """Determine if the stock price will go up or down the next day."""
    data['Next Day Return'] = data['Close'].shift(-1) - data['Close']
    data['Uptrend'] = (data['Next Day Return'] > 0).astype(int)
    return data

Bayesian Calculations


*   Compute prior: Proportion of days when the stock increased.
*   Compute likelihoods
*   Compute evidence (normalization factor): Ensures probabilities sum to 1.
*   Compute posterior: Bayesian formula to update prior with observed data.



1.   Return: A probability estimate (P(Uptrend | Data))







In [ ]:
def bayesian_update(data):
    """Apply Bayesian updating to estimate P(Price Increase | Data) using volume-weighted likelihoods."""
    total_days = len(data.dropna())
    up_days = data['Uptrend'].sum()
    down_days = total_days - up_days

    prior = up_days / total_days if total_days > 0 else 0.5  # Prior probability

    # Volume-weighted likelihood calculation
    avg_volume_up = data.loc[data['Uptrend'] == 1, 'Volume'].mean()
    avg_volume_down = data.loc[data['Uptrend'] == 0, 'Volume'].mean()
    total_avg_volume = data['Volume'].mean()

    likelihood_up = avg_volume_up / total_avg_volume if total_avg_volume > 0 else 0.5
    likelihood_down = avg_volume_down / total_avg_volume if total_avg_volume > 0 else 0.5

    # Normalizing constant P(Data)
    evidence = (likelihood_up * prior) + (likelihood_down * (1 - prior))

    # Posterior: P(Uptrend | Data)
    posterior = (likelihood_up * prior) / evidence if evidence > 0 else 0

    return posterior

Expected Utility of making a trade



1.   Return: Utility value and boolean indicating whether it exceeds the risk threshold


In [ ]:
def expected_utility(posterior, expected_profit, expected_loss, risk_threshold):
    """Calculate the expected utility for making a trade decision."""
    utility = (posterior * expected_profit) - ((1 - posterior) * expected_loss)
    return utility, utility > risk_threshold

Make a Trade decision based on the Posterior Prob. and Expected Utility


*   Decision Rules:
*   BUY if posterior > 0.7 and utility is positive.
*   SELL if posterior < 0.3 (high probability of decline).
*   HOLD otherwise.



1.   Return: Trade Action





In [ ]:
def make_trade_decision(posterior, expected_profit, expected_loss, risk_threshold, threshold=0.7):
    """Make a trading decision based on the posterior probability and expected utility."""
    utility, trade_decision = expected_utility(posterior, expected_profit, expected_loss, risk_threshold)

    if posterior > threshold and trade_decision:
        return "BUY"
    elif posterior < (1 - threshold):
        return "SELL"
    else:
        return "HOLD"

# ***Running things here***

In [ ]:
if __name__ == "__main__":
    print("Please upload your stock data ZIP file containing a CSV.")
    stock_data = load_data()
    expected_profit = float(input("Enter the expected profit percentage (e.g., 0.05 for 5%): "))
    expected_loss = float(input("Enter the expected loss percentage (e.g., 0.03 for 3%): "))
    risk_threshold = float(input("Enter the risk threshold for trading decision: "))

    stock_data = calculate_indicators(stock_data)
    stock_data = calculate_price_movement(stock_data)

    posterior_prob = bayesian_update(stock_data)
    decision = make_trade_decision(posterior_prob, expected_profit, expected_loss, risk_threshold)

    print(f"Posterior Probability of Uptrend: {posterior_prob:.4f}")
    print(f"Trading Decision: {decision}")